### This serves as the document for various versions of the recursion algorithm for digging out the flows of interest

In [13]:
from brightway2 import*
from bw2data import get_activity
from collections import defaultdict

#### set up project

In [4]:
projects.set_current("trial recursion dig")
projects.current

'trial recursion dig'

In [6]:
bw2setup()

Creating default biosphere



Writing activities to SQLite3 database:


Applying strategy: normalize_units
Applying strategy: drop_unspecified_subcategories
Applied 2 strategies in 0.01 seconds


0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Title: Writing activities to SQLite3 database:
  Started: 01/14/2020 10:02:29
  Finished: 01/14/2020 10:02:29
  Total time elapsed: 00:00:00
  CPU %: 98.30
  Memory %: 1.91
Created database: biosphere3
Creating default LCIA methods

Applying strategy: normalize_units
Applying strategy: set_biosphere_type
Applying strategy: drop_unspecified_subcategories
Applying strategy: link_iterable_by_fields
Applied 4 strategies in 1.66 seconds
Wrote 850 LCIA methods with 220699 characterization factors
Creating core data migrations



#### import Ecoinvent

In [7]:
#create an object to import database
ei35_cutoff=SingleOutputEcospold2Importer(
    "/Users/Qingshi/Downloads/ecoinvent 3.5_cutoff_ecoSpold02/datasets",
    "ecoinvent 3.5 cutoff")
ei35_cutoff.apply_strategies()
ei35_cutoff.statistics()

Extracting XML data from 16022 datasets
Extracted 16022 datasets in 63.71 seconds
Applying strategy: normalize_units
Applying strategy: update_ecoinvent_locations
Applying strategy: remove_zero_amount_coproducts
Applying strategy: remove_zero_amount_inputs_with_no_activity
Applying strategy: remove_unnamed_parameters
Applying strategy: es2_assign_only_product_with_amount_as_reference_product
Applying strategy: assign_single_product_as_activity
Applying strategy: create_composite_code
Applying strategy: drop_unspecified_subcategories
Applying strategy: fix_ecoinvent_flows_pre35
Applying strategy: drop_temporary_outdated_biosphere_flows
Applying strategy: link_biosphere_by_flow_uuid
Applying strategy: link_internal_technosphere_by_composite_code
Applying strategy: delete_exchanges_missing_activity
Applying strategy: delete_ghost_exchanges
Applying strategy: remove_uncertainty_from_negative_loss_exchanges
Applying strategy: fix_unreasonably_high_lognormal_uncertainties
Applying strategy: 

(16022, 544735, 0)

In [8]:
#to actually write the database into hard disc
ei35_cutoff.write_database()

Writing activities to SQLite3 database:
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:54


Title: Writing activities to SQLite3 database:
  Started: 01/14/2020 10:06:56
  Finished: 01/14/2020 10:07:50
  Total time elapsed: 00:00:54
  CPU %: 90.90
  Memory %: 16.07
Created database: ecoinvent 3.5 cutoff


Brightway2 SQLiteBackend: ecoinvent 3.5 cutoff

In [9]:
db=Database("ecoinvent 3.5 cutoff") #to create a database object

#### look for an activity to test on

In [10]:
waste_act=db.search('scrap')[0]
waste_act

'market for scrap steel' (kilogram, RoW, None)

#### ver 3

In [11]:
def recur_dig3(activity,keyword,depth,glo_store_dict):
    """This function applies recursion to dig out the activities associated with recycling"""
    
    ##parse activity
    if isinstance(activity, tuple):
        activity = get_activity(activity) #the activity tuple should be (database,code): seems only necessary for initial input
    

    if keyword in activity['name']:
        glo_store_dict[depth].add(activity['name']) #same for base case and recursion step
        
    ##Recursion step
    if depth>0:
        for exc in activity.technosphere():
            recur_dig3(exc.input,keyword,depth-1,glo_store_dict)

In [14]:
glo_store_dict=defaultdict(set)
recur_dig3(('ecoinvent 3.5 cutoff','87fc24db0970fbff77e602f2b00b4cf9'),keyword='Recycled Content',depth=5,glo_store_dict=glo_store_dict)

In [15]:
glo_store_dict

defaultdict(set,
            {3: {'iron scrap, sorted, pressed, Recycled Content cut-off'},
             0: {'digester sludge, Recycled Content cut-off',
              'iron scrap, unsorted, Recycled Content cut-off',
              'waste newspaper, Recycled Content cut-off',
              'waste packaging paper, Recycled Content cut-off',
              'waste paper, unsorted, Recycled Content cut-off'},
             1: {'metalliferous hydroxide sludge, Recycled Content cut-off'}})

In [16]:
##Information regarding LCI:
https://chris.mutel.org/calculating-inventory-flows.html

SyntaxError: invalid syntax (<ipython-input-16-074afe2a99e1>, line 2)

#### ver 2

In [17]:
def recur_dig2(activity,keyword,depth,glo_store_dict):
    """This function applies recursion to dig out the activities associated with recycling"""
    
    ##parse activity
    if isinstance(activity, tuple):
        activity = get_activity(activity) #the activity tuple should be (database,code): seems only necessary for initial input
    

    if keyword in activity['name']:
        glo_store_dict[depth].append({activity['name']:activity['production amount']}) #same for base case and recursion step
        
    ##Recursion step
    if depth>0:
        for exc in activity.technosphere():
            recur_dig2(exc.input,keyword,depth-1,glo_store_dict)

In [18]:
glo_store_dict=defaultdict(list)
recur_dig2(('ecoinvent 3.5 cutoff','87fc24db0970fbff77e602f2b00b4cf9'),keyword='Recycled Content',depth=5,glo_store_dict=glo_store_dict)

In [19]:
glo_store_dict

defaultdict(list,
            {3: [{'iron scrap, sorted, pressed, Recycled Content cut-off': 1.0}],
             0: [{'digester sludge, Recycled Content cut-off': 1.0},
              {'waste newspaper, Recycled Content cut-off': 1.0},
              {'waste packaging paper, Recycled Content cut-off': 1.0},
              {'digester sludge, Recycled Content cut-off': 1.0},
              {'waste newspaper, Recycled Content cut-off': 1.0},
              {'waste packaging paper, Recycled Content cut-off': 1.0},
              {'waste packaging paper, Recycled Content cut-off': 1.0},
              {'waste packaging paper, Recycled Content cut-off': 1.0},
              {'waste packaging paper, Recycled Content cut-off': 1.0},
              {'waste packaging paper, Recycled Content cut-off': 1.0},
              {'waste packaging paper, Recycled Content cut-off': 1.0},
              {'waste packaging paper, Recycled Content cut-off': 1.0},
              {'iron scrap, unsorted, Recycled Content cu

#### ver 1

In [20]:
def recur_dig(activity,keyword,depth):
    """This function applies recursion to dig out the activities associated with recycling"""
    
    ##parse activity
    if isinstance(activity, tuple):
        activity = get_activity(activity) #the activity tuple should be (database,code): seems only necessary for initial input
    
    ##initiate an empty list
    act_of_interest=[]
    if keyword in activity['name']:
        act_of_interest.append(activity['name'])
        
    ##Base case
    if depth==0:
        return {depth:act_of_interest}
    
    ##Recursion step
    else:
        return {depth: act_of_interest+[recur_dig(exc.input,keyword,depth-1) for exc in activity.technosphere()]}
        

In [21]:
result_of_interest= (recur_dig(('ecoinvent 3.5 cutoff','21052cc2ea2110e327dc5828d45b3c83'),keyword='electricity',depth=5))


In [22]:
result_of_interest

{5: [{4: [{3: [{2: [{1: [{0: []},
          {0: ['transport, freight train, electricity']}]},
        {1: [{0: []}, {0: ['transport, freight train, electricity']}]}]},
      {2: [{1: [{0: []}, {0: []}, {0: []}, {0: []}, {0: []}, {0: []}]},
        {1: ['transport, freight train, electricity',
          {0: []},
          {0: []},
          {0: ['market group for electricity, medium voltage']},
          {0: []},
          {0: []},
          {0: []},
          {0: []}]},
        {1: [{0: []}, {0: []}, {0: []}, {0: []}, {0: []}, {0: []}, {0: []}]}]},
      {2: [{1: [{0: []}, {0: []}, {0: []}, {0: []}, {0: []}, {0: []}]}]},
      {2: [{1: ['transport, freight train, electricity',
          {0: []},
          {0: []},
          {0: []},
          {0: []},
          {0: ['market for electricity, high voltage, for Swiss Federal Railways']},
          {0: []},
          {0: []},
          {0: ['market for electricity, medium voltage']},
          {0: ['market for electricity, medium voltage']